## Instalacion de librerias de langchain

In [2]:
%pip install --upgrade --quiet langchain langchain-openai langchain_google_community langchain_community langchainhub pypdf faiss-cpu tiktoken google-search-results wikipedia

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: wikipedia is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: google-search-results is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\Seba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

#from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent

langchain.debug = True
os.environ["OPENAI_API_KEY"] = ""

## Subida de documentos, embedding y base de datos vectorial

In [4]:
loader = PyPDFLoader("C:/Users/Seba/capstone/desarrollo/backend/api/celula_unidad_funcional_de_los_seres_vivos.pdf")


pages = loader.load_and_split()

# Funcion para crear embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

# Creamos una base de datos vectorial a partir de las páginas del documento
faiss_index = FAISS.from_documents(pages, embeddings)
retriever = faiss_index.as_retriever()

## Creacion del modelo LLM con mi Template Prompt

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
           """Eres un un profesor experto Ciencias de la prueba PAES, PTA, PSU de Chile, y los alumnos vienen a solicitarte preguntas de 5 alternativas para poder poner a prueba sus conocimientos. Entrega las preguntas y respuesta en formato json, asi:
pregunta": "Texto de la pregunta",
                "opciones": [
                    {{"texto": "Opción 1", "es_correcta": none}},
                    {{"texto": "Opción 2", "es_correcta": none}},
                    {{"texto": "Opción 3", "es_correcta": none}},
                    {{"texto": "Opción 4", "es_correcta": none}},
                    {{"texto": "Opción 5", "es_correcta": none}}
                ]
Usa la siguiente informacion para elavorar tus preguntas.
{context}.
Solo entrega el json.
""",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Creacion de Tool para distinta materia

#### Esto permite que en cada tool se alimente de distintos documentos al realizar la pregunta

In [6]:
retriever_tool = create_retriever_tool(
    retriever, #No olvidar que este retriver tiene solo el documento de las celulas
    "Materia_Ciencias_Biologia",
    "Usa este recurso para preguntas que correspondan al campo de la Biologia.",
)

tools = [retriever_tool]

### Creacion de modelo y agente

In [7]:
model = ChatOpenAI(temperature=0.3, model="gpt-4o", seed=211)
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
#Lista de herramientas, cono solo una, el documento de la celula, por ahora
tools

[Tool(name='Materia_Ciencias_Biologia', description='Usa este recurso para preguntas que correspondan al campo de la Biologia.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001CDEBF39120>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CD84DCD420>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001CDEC0FCAF0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CD84DCD420>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_conten

In [9]:
user_input = "Cual es el mayor secreto de harry potter?"
context_info = ""
# Ejecuta la consulta usando invoke
response = agent_executor.invoke({"input": user_input, "context": context_info})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Cual es el mayor secreto de harry potter?",
  "context": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain ru

## Creacion de Tools personalizado

In [10]:
from langchain_core.tools import tool

@tool
def secretos_harry_potter() -> str:
    """Entrega el mayor secreto de Harry Potter."""
    return "El mayor secreto de Harry Potter es que sueña con empanadas de pino."


#### Actualizamos tools y agente

In [11]:
tools = [retriever_tool,secretos_harry_potter]
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
#Aseguramos que la nueva informacion de ingreso como herramienta, y aun sigue tomando el template determinadom anteriormente
user_input = "Cual es el mayor secreto de harry potter?"
context_info = ""
# Ejecuta la consulta usando invoke
response = agent_executor.invoke({"input": user_input, "context": context_info})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Cual es el mayor secreto de harry potter?",
  "context": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain ru

### Vamos a entregar cuiestionarios previos por tipo de evaluacion al modelo para que les sirva de referencia

In [1]:
#Repetimos el paso de crear un nuevo retriever, pero esta vez de los cuestionarios